In [ ]:
import torch
import pandas as pd
import numpy as np
data = pd.read_csv()

In [ ]:
from sklearn import preprocessing

In [ ]:
data[] = preprocessing.scale(data[])

In [ ]:
target = data[]

In [ ]:
y.replace(to_replace = target[target>=0.80], value = int(2),inplace=True)
y.replace(to_replace = target[target>=0.60], value = int(1),inplace=True)
y.replace(to_replace = target[target<0.60], value = int(0),inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,y_test,y_train,y_test = train_test_split(features,target,test_size=0.2)

In [ ]:
xtrain = torch.from_numpy(x_train.values).float()
x_test = torch.from_numpy(x_test.values).float()

ytrain = torch.from_numpy(y_train.values).view(1,-1)[0].long()
ytest = torch.from_numpy(y_test.values).view(1,-1)[0].long() # view = reshape

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# creating custom nn layers
input_size = xtrain.shape[1]
output_size = len(target[''].unique())

In [ ]:
class Net(nn.Module):
    
    def __init__(self,hidden_size,activation_fn = 'relu', apply_dropout = False):
        super(Net,self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
        self.hidden_size = hidden_size
        self.activation_fn = activation_fn
        
        self.dropout = None
        if apply_dropout:
            self.dropout = nn.Dropout(0.2)
            
    def forward(self, x): #foward pass
        activation_fn = None
        if self.activation_fn == 'sigmoid':
            activation_fn = F.torch.sigmoid
        elif self.activation_fn == 'tanh':
            activation_fn = F.torch.tanh
        elif self.activation_fn == 'relu':
            activation_fn = F.relu
            
        x = activation_fn(self.fc1(x))
        x = activation_fn(self.fc2(x))
        
        if self.dropout !=None:
            x = self.dropout(x)
            
        x = self.fc3(x)
        # calculates the log(softmax()) - used in conjunction with NLLloss()
        # faster and more numerically stable than softmax
        return F.log_softmax(x,dim = -1) 
    
    

In [ ]:
import torch.optim as optim

In [ ]:
def train_and_eval(model,learn_rate=0.001):
    epoch_data=[]
    epochs=1001
    
    optimizer = optim.Adam(model.parameters(),lr=learn_rate)
    
    loss_fn = nn.NLLLoss()
    
    test_accuracy = 0.0
    for epoch in range(1, epochs):
        
        optimizer.zero_grad()
        
        model.train()
        ypred = model(xtrain)
        
        loss = loss_fn(ypred,ytrain)
        loss.backward()
        
        optimizer.step()
        
        model.eval()
        
        ypred_t = model(xtest)
        loss_test = loss_fn(ypred_t,ytest)
        _, pred = ypred_t .data.max(1)
        
        test_accuracy = pred.eq(ytest.data).sum().item()/ytest.values.size
        
        epoch_data.append([epoch,loss.data.item(), loss_test.data.item(), test_accuracy])
        
        # verbose every 100
        if epoch%100 == 0:
            print(...)
            
        return {'model':model,
               'epoch_data':epoch_data,
               'num_epochs':num_epochs,
               'optimizer':optimizer,
               'loss_fn':loss_fn,
               'test_acc':test_accuracy,
               }

In [ ]:
net = Net(hidden_size=3,activation_fn='sigmoid',apply_dropout=False)

result_3_sigmoid = train_and_eval(net)

In [ ]:
net = Net(hidden_size=3,activation_fn='sigmoid',apply_dropout=True)

result_3_sigmoid_do = train_and_eval(net)

In [ ]:
net = Net(hidden_size=50,activation_fn='tanh',apply_dropout=True)

result_1000_tanh_do = train_and_eval(net)